![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB01_Regression.ipynb)

# NB01 - Regressão e Séries Temporais (FP&A Banco BV)

**Objetivo:** aplicar regressão e séries temporais em dados sintéticos de FP&A do BV, de forma simples e prática.
**Neste notebook, vamos comparar baseline e modelos com split temporal, usando MAE e RMSE.**
**Foco total em leitura de negócio, sem complexidade de programação.**


## 1) Setup simples

### O que vamos fazer
Importar bibliotecas e definir apenas variáveis básicas.

### Por que isso importa para FP&A
Menos código técnico, mais foco na análise.

### O que você deve ver no output
Sem erro de import.


In [ ]:
# RUN_ME 1 - imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX


## 2) PARTE A - Regressão com dataset linear (X -> y)

### O que vamos fazer
Carregar dataset linear e comparar baseline + modelos.

### Por que isso importa para FP&A
Ajuda a prever originação a partir de drivers de negócio.


In [ ]:
# RUN_ME 2 - carregar dataset linear (já pronto para uso)
url_linear = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_linear.csv'

df_lin = pd.read_csv(url_linear)
df_lin['ds'] = pd.to_datetime(df_lin['ds'])

df_lin.head()


In [ ]:
# RUN_ME 3 - inspeção rápida
print('shape:', df_lin.shape)
print('colunas:', list(df_lin.columns))
print('
dtypes:')
print(df_lin.dtypes)


### 2.1 Visualização inicial

### O que vamos fazer
Olhar `y` no tempo e relação `selic` vs `y`.

### O que você deve ver no output
Um gráfico de linha e um gráfico de dispersão.


In [ ]:
# RUN_ME 4 - linha de y no tempo
plt.figure(figsize=(10, 4))
plt.plot(df_lin['ds'], df_lin['y'], linewidth=2)
plt.title('PARTE A - y ao longo do tempo')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# RUN_ME 5 - scatter selic vs y
plt.figure(figsize=(6, 4))
plt.scatter(df_lin['selic'], df_lin['y'], alpha=0.75)
plt.title('PARTE A - SELIC vs y')
plt.xlabel('SELIC')
plt.ylabel('y (R$ milhões)')
plt.grid(alpha=0.3)
plt.show()


### 2.2 Split temporal e baseline

### O que vamos fazer
Separar treino/teste no tempo (80/20), sem embaralhar.

### Por que isso importa para FP&A
Treinar no passado e testar no futuro.


In [ ]:
# RUN_ME 6 - preparar X/y e split temporal
features = ['selic', 'desemprego', 'ltv_medio', 'spread', 'marketing', 'mix_auto', 'mes']

split_idx = int(len(df_lin) * 0.8)
train_lin = df_lin.iloc[:split_idx].copy()
test_lin = df_lin.iloc[split_idx:].copy()

X_train_lin = train_lin[features]
X_test_lin = test_lin[features]
y_train_lin = train_lin['y']
y_test_lin = test_lin['y']

print('Treino:', X_train_lin.shape, 'Teste:', X_test_lin.shape)
print('Período treino:', train_lin['ds'].min().date(), '->', train_lin['ds'].max().date())
print('Período teste :', test_lin['ds'].min().date(), '->', test_lin['ds'].max().date())


In [ ]:
# RUN_ME 7 - baseline naive (último valor do treino)
baseline_value_lin = y_train_lin.iloc[-1]
pred_baseline_lin = np.repeat(baseline_value_lin, len(y_test_lin))

mae_baseline_lin = mean_absolute_error(y_test_lin, pred_baseline_lin)
rmse_baseline_lin = np.sqrt(mean_squared_error(y_test_lin, pred_baseline_lin))

print('Baseline MAE :', round(mae_baseline_lin, 3))
print('Baseline RMSE:', round(rmse_baseline_lin, 3))


### 2.3 Modelos de regressão (simples e direto)

### O que vamos fazer
Treinar 6 modelos e comparar no mesmo teste.

### O que você deve ver no output
Tabela ordenada por RMSE.


In [ ]:
# RUN_ME 8 - Linear Regression
model_lr = LinearRegression()
model_lr.fit(X_train_lin, y_train_lin)
pred_lr = model_lr.predict(X_test_lin)

mae_lr = mean_absolute_error(y_test_lin, pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test_lin, pred_lr))


In [ ]:
# RUN_ME 9 - Ridge
model_ridge = Ridge(alpha=1.0, random_state=42)
model_ridge.fit(X_train_lin, y_train_lin)
pred_ridge = model_ridge.predict(X_test_lin)

mae_ridge = mean_absolute_error(y_test_lin, pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test_lin, pred_ridge))


In [ ]:
# RUN_ME 10 - ElasticNet
model_en = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)
model_en.fit(X_train_lin, y_train_lin)
pred_en = model_en.predict(X_test_lin)

mae_en = mean_absolute_error(y_test_lin, pred_en)
rmse_en = np.sqrt(mean_squared_error(y_test_lin, pred_en))


In [ ]:
# RUN_ME 11 - DecisionTree
model_tree = DecisionTreeRegressor(max_depth=4, random_state=42)
model_tree.fit(X_train_lin, y_train_lin)
pred_tree = model_tree.predict(X_test_lin)

mae_tree = mean_absolute_error(y_test_lin, pred_tree)
rmse_tree = np.sqrt(mean_squared_error(y_test_lin, pred_tree))


In [ ]:
# RUN_ME 12 - RandomForest
model_rf = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)
model_rf.fit(X_train_lin, y_train_lin)
pred_rf = model_rf.predict(X_test_lin)

mae_rf = mean_absolute_error(y_test_lin, pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test_lin, pred_rf))


In [ ]:
# RUN_ME 13 - GradientBoosting
model_gbm = GradientBoostingRegressor(random_state=42)
model_gbm.fit(X_train_lin, y_train_lin)
pred_gbm = model_gbm.predict(X_test_lin)

mae_gbm = mean_absolute_error(y_test_lin, pred_gbm)
rmse_gbm = np.sqrt(mean_squared_error(y_test_lin, pred_gbm))


In [ ]:
# RUN_ME 14 - tabela comparativa (PARTE A)
results_a = pd.DataFrame([
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_lin, 'rmse': rmse_baseline_lin},
    {'modelo': 'LinearRegression', 'mae': mae_lr, 'rmse': rmse_lr},
    {'modelo': 'Ridge', 'mae': mae_ridge, 'rmse': rmse_ridge},
    {'modelo': 'ElasticNet', 'mae': mae_en, 'rmse': rmse_en},
    {'modelo': 'DecisionTree', 'mae': mae_tree, 'rmse': rmse_tree},
    {'modelo': 'RandomForest', 'mae': mae_rf, 'rmse': rmse_rf},
    {'modelo': 'GradientBoosting', 'mae': mae_gbm, 'rmse': rmse_gbm},
]).sort_values('rmse').reset_index(drop=True)

results_a


In [ ]:
# RUN_ME 15 - real vs previsto (baseline, melhor linear, melhor não-linear)
best_linear_name = results_a[results_a['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']
best_nonlinear_name = results_a[results_a['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']

pred_map_a = {
    'LinearRegression': pred_lr,
    'Ridge': pred_ridge,
    'ElasticNet': pred_en,
    'DecisionTree': pred_tree,
    'RandomForest': pred_rf,
    'GradientBoosting': pred_gbm
}

plt.figure(figsize=(11, 4))
plt.plot(test_lin['ds'], y_test_lin.values, label='Real', color='black', linewidth=2)
plt.plot(test_lin['ds'], pred_baseline_lin, label='Baseline_Naive', linewidth=1.8)
plt.plot(test_lin['ds'], pred_map_a[best_linear_name], label=f'Melhor linear: {best_linear_name}', linewidth=1.8)
plt.plot(test_lin['ds'], pred_map_a[best_nonlinear_name], label=f'Melhor não-linear: {best_nonlinear_name}', linewidth=1.8)
plt.title('PARTE A - Real vs Previsto (teste)')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# RUN_ME 16 - feature importance (RandomForest e GradientBoosting)
rf_imp = pd.Series(model_rf.feature_importances_, index=features).sort_values(ascending=True)
gbm_imp = pd.Series(model_gbm.feature_importances_, index=features).sort_values(ascending=True)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.barh(rf_imp.index, rf_imp.values)
plt.title('Importância - RandomForest')
plt.grid(axis='x', alpha=0.3)

plt.subplot(1, 2, 2)
plt.barh(gbm_imp.index, gbm_imp.values)
plt.title('Importância - GradientBoosting')
plt.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


**Importante:** importância de variável ajuda a priorizar discussão, mas **não prova causalidade**.


## 3) PARTE B - Mesmo fluxo no dataset não-linear

### O que vamos fazer
Repetir o mesmo processo no dataset não-linear.

### Por que isso importa para FP&A
Quando há limiar/interação, modelos não-lineares costumam performar melhor.


In [ ]:
# RUN_ME 17 - carregar dataset não-linear
url_nlin = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_nonlinear.csv'

df_nlin = pd.read_csv(url_nlin)
df_nlin['ds'] = pd.to_datetime(df_nlin['ds'])

df_nlin.head()


In [ ]:
# RUN_ME 18 - split temporal e baseline (PARTE B)
split_idx_b = int(len(df_nlin) * 0.8)
train_nlin = df_nlin.iloc[:split_idx_b].copy()
test_nlin = df_nlin.iloc[split_idx_b:].copy()

X_train_nlin = train_nlin[features]
X_test_nlin = test_nlin[features]
y_train_nlin = train_nlin['y']
y_test_nlin = test_nlin['y']

baseline_value_nlin = y_train_nlin.iloc[-1]
pred_baseline_nlin = np.repeat(baseline_value_nlin, len(y_test_nlin))

mae_baseline_nlin = mean_absolute_error(y_test_nlin, pred_baseline_nlin)
rmse_baseline_nlin = np.sqrt(mean_squared_error(y_test_nlin, pred_baseline_nlin))

print('Baseline RMSE (não-linear):', round(rmse_baseline_nlin, 3))


In [ ]:
# RUN_ME 19 - treinar modelos no dataset não-linear
# Lineares
model_lr_b = LinearRegression().fit(X_train_nlin, y_train_nlin)
model_ridge_b = Ridge(alpha=1.0, random_state=42).fit(X_train_nlin, y_train_nlin)
model_en_b = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000).fit(X_train_nlin, y_train_nlin)

# Não-lineares
model_tree_b = DecisionTreeRegressor(max_depth=4, random_state=42).fit(X_train_nlin, y_train_nlin)
model_rf_b = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42).fit(X_train_nlin, y_train_nlin)
model_gbm_b = GradientBoostingRegressor(random_state=42).fit(X_train_nlin, y_train_nlin)

# Predições
pred_lr_b = model_lr_b.predict(X_test_nlin)
pred_ridge_b = model_ridge_b.predict(X_test_nlin)
pred_en_b = model_en_b.predict(X_test_nlin)
pred_tree_b = model_tree_b.predict(X_test_nlin)
pred_rf_b = model_rf_b.predict(X_test_nlin)
pred_gbm_b = model_gbm_b.predict(X_test_nlin)


In [ ]:
# RUN_ME 20 - métricas (PARTE B)
results_b = pd.DataFrame([
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_nlin, 'rmse': rmse_baseline_nlin},
    {'modelo': 'LinearRegression', 'mae': mean_absolute_error(y_test_nlin, pred_lr_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_lr_b))},
    {'modelo': 'Ridge', 'mae': mean_absolute_error(y_test_nlin, pred_ridge_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_ridge_b))},
    {'modelo': 'ElasticNet', 'mae': mean_absolute_error(y_test_nlin, pred_en_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_en_b))},
    {'modelo': 'DecisionTree', 'mae': mean_absolute_error(y_test_nlin, pred_tree_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_tree_b))},
    {'modelo': 'RandomForest', 'mae': mean_absolute_error(y_test_nlin, pred_rf_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_rf_b))},
    {'modelo': 'GradientBoosting', 'mae': mean_absolute_error(y_test_nlin, pred_gbm_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_gbm_b))},
]).sort_values('rmse').reset_index(drop=True)

results_b


In [ ]:
# RUN_ME 21 - comparar melhor linear vs melhor não-linear (PARTE B)
best_linear_b = results_b[results_b['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']
best_nonlinear_b = results_b[results_b['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']

pred_map_b = {
    'LinearRegression': pred_lr_b,
    'Ridge': pred_ridge_b,
    'ElasticNet': pred_en_b,
    'DecisionTree': pred_tree_b,
    'RandomForest': pred_rf_b,
    'GradientBoosting': pred_gbm_b,
}

plt.figure(figsize=(11, 4))
plt.plot(test_nlin['ds'], y_test_nlin.values, label='Real', color='black', linewidth=2)
plt.plot(test_nlin['ds'], pred_map_b[best_linear_b], label=f'Melhor linear: {best_linear_b}', linewidth=1.8)
plt.plot(test_nlin['ds'], pred_map_b[best_nonlinear_b], label=f'Melhor não-linear: {best_nonlinear_b}', linewidth=1.8)
plt.title('PARTE B - Melhor linear vs melhor não-linear (teste)')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


## 4) PARTE C - Séries temporais (t -> y)

### O que vamos fazer
Comparar Baseline, ETS, ARIMA, ARIMAX e Prophet (opcional).

### Por que isso importa para FP&A
Forecast de série temporal é comum quando o histórico já contém sinal forte.


In [ ]:
# RUN_ME 22 - carregar dataset de séries temporais
url_ts = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_timeseries.csv'

df_ts = pd.read_csv(url_ts)
df_ts['ds'] = pd.to_datetime(df_ts['ds'])

df_ts.head()


In [ ]:
# RUN_ME 23 - plot da série
plt.figure(figsize=(10, 4))
plt.plot(df_ts['ds'], df_ts['y'], linewidth=2)
plt.title('PARTE C - Série temporal y')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# RUN_ME 24 - split temporal (últimos 12 meses no teste)
holdout = 12
train_ts = df_ts.iloc[:-holdout].copy()
test_ts = df_ts.iloc[-holdout:].copy()

y_train_ts = train_ts['y'].values
y_test_ts = test_ts['y'].values

print('Treino:', train_ts.shape, 'Teste:', test_ts.shape)


In [ ]:
# RUN_ME 25 - baseline naive
pred_naive_ts = np.repeat(y_train_ts[-1], len(y_test_ts))
mae_naive_ts = mean_absolute_error(y_test_ts, pred_naive_ts)
rmse_naive_ts = np.sqrt(mean_squared_error(y_test_ts, pred_naive_ts))

print('Baseline MAE :', round(mae_naive_ts, 3))
print('Baseline RMSE:', round(rmse_naive_ts, 3))


In [ ]:
# RUN_ME 26 - ETS (Holt-Winters)
ets = ExponentialSmoothing(train_ts['y'], trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)
pred_ets = ets.forecast(len(test_ts)).values

mae_ets = mean_absolute_error(y_test_ts, pred_ets)
rmse_ets = np.sqrt(mean_squared_error(y_test_ts, pred_ets))

print('ETS MAE :', round(mae_ets, 3))
print('ETS RMSE:', round(rmse_ets, 3))


In [ ]:
# RUN_ME 27 - ARIMA
arima = SARIMAX(train_ts['y'], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
pred_arima = arima.forecast(steps=len(test_ts)).values

mae_arima = mean_absolute_error(y_test_ts, pred_arima)
rmse_arima = np.sqrt(mean_squared_error(y_test_ts, pred_arima))

print('ARIMA MAE :', round(mae_arima, 3))
print('ARIMA RMSE:', round(rmse_arima, 3))


In [ ]:
# RUN_ME 28 - ARIMAX (com selic)
arimax = SARIMAX(
    train_ts['y'],
    exog=train_ts[['selic']],
    order=(1, 1, 1),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

pred_arimax = arimax.forecast(steps=len(test_ts), exog=test_ts[['selic']]).values

mae_arimax = mean_absolute_error(y_test_ts, pred_arimax)
rmse_arimax = np.sqrt(mean_squared_error(y_test_ts, pred_arimax))

print('ARIMAX MAE :', round(mae_arimax, 3))
print('ARIMAX RMSE:', round(rmse_arimax, 3))


### 4.1 Prophet (opcional)

Se funcionar no ambiente, ótimo. Se não funcionar, siga com ETS/ARIMA/ARIMAX sem problema.


In [ ]:
# RUN_ME 29 (opcional) - instalar/importar Prophet
import subprocess, sys
prophet_ok = False

try:
    from prophet import Prophet
    prophet_ok = True
    print('Prophet já disponível.')
except Exception:
    print('Tentando instalar Prophet...')
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'prophet', '-q'])
        from prophet import Prophet
        prophet_ok = True
        print('Prophet instalado com sucesso.')
    except Exception as e:
        print('Prophet indisponível nesta sessão.')
        print('Erro:', e)


In [ ]:
# RUN_ME 30 (opcional) - previsão com Prophet
pred_prophet = None
mae_prophet = None
rmse_prophet = None

if prophet_ok:
    train_prophet = train_ts[['ds', 'y', 'selic', 'evento']].copy()
    test_prophet = test_ts[['ds', 'selic', 'evento']].copy()

    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    m.add_regressor('selic')
    m.add_regressor('evento')
    m.fit(train_prophet)

    fcst = m.predict(test_prophet)
    pred_prophet = fcst['yhat'].values

    mae_prophet = mean_absolute_error(y_test_ts, pred_prophet)
    rmse_prophet = np.sqrt(mean_squared_error(y_test_ts, pred_prophet))

    print('Prophet MAE :', round(mae_prophet, 3))
    print('Prophet RMSE:', round(rmse_prophet, 3))
else:
    print('Prophet não executado.')


In [ ]:
# RUN_ME 31 - tabela final PARTE C
results_c = [
    {'modelo': 'Baseline_Naive', 'mae': mae_naive_ts, 'rmse': rmse_naive_ts},
    {'modelo': 'ETS_HoltWinters', 'mae': mae_ets, 'rmse': rmse_ets},
    {'modelo': 'ARIMA', 'mae': mae_arima, 'rmse': rmse_arima},
    {'modelo': 'ARIMAX_selic', 'mae': mae_arimax, 'rmse': rmse_arimax},
]

if pred_prophet is not None:
    results_c.append({'modelo': 'Prophet', 'mae': mae_prophet, 'rmse': rmse_prophet})

results_c = pd.DataFrame(results_c).sort_values('rmse').reset_index(drop=True)
results_c


In [ ]:
# RUN_ME 32 - gráfico final (real vs previsões)
plt.figure(figsize=(11, 4))
plt.plot(test_ts['ds'], y_test_ts, label='Real', color='black', linewidth=2)
plt.plot(test_ts['ds'], pred_naive_ts, label='Baseline_Naive', linewidth=1.8)
plt.plot(test_ts['ds'], pred_ets, label='ETS_HoltWinters', linewidth=1.8)
plt.plot(test_ts['ds'], pred_arimax, label='ARIMAX_selic', linewidth=1.8)

if pred_prophet is not None:
    plt.plot(test_ts['ds'], pred_prophet, label='Prophet', linewidth=1.8)

plt.title('PARTE C - Real vs Previsões (teste)')
plt.xlabel('Data')
plt.ylabel('y (R$ milhões)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


## 5) Conclusão rápida

- **X -> y (regressão):** melhor quando os drivers são conhecidos e úteis para cenário.
- **t -> y (série temporal):** melhor quando o histórico da própria série é forte.
- Em ambos os casos: validar no tempo e comparar com baseline.


## 6) Checklist FP&A

- Baseline: compare sempre com um método simples antes de escolher modelo.
- Split temporal: treino no passado, teste no futuro (sem embaralhar).
- Métricas: acompanhar MAE e RMSE em teste.
- Governança: registrar versão do modelo, dados e período de treino.
- Monitoramento: acompanhar performance e recalibrar quando necessário.
